In [1]:
cd /content/drive/MyDrive/졸업프로젝트/

/content/drive/MyDrive/졸업프로젝트


In [2]:
!pip install praat-parselmouth
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.impute import SimpleImputer
import sys
import os
from parselmouth.praat import call

def pitch_bounds(sound):
  # measure pitch ceiling and floor
  broad_pitch = sound.to_pitch_ac(
      None, 50, 15, True, 0.03, 0.45, 0.01, 0.35, 0.14, 500
  )
  # get mean pitch
  broad_mean_f0: float = call(
      broad_pitch, "Get mean", 0, 0, "hertz"
  )

  if broad_mean_f0 > 170:
    pitch_floor = 100
    pitch_ceiling = 500
  elif broad_mean_f0 < 170:
    pitch_floor = 50
    pitch_ceiling = 300
  else:
    pitch_floor = 50
    pitch_ceiling = 500
  return pitch_floor, pitch_ceiling

In [4]:
def formant_max(sound):
  try:
    pitch_floor, pitch_ceiling = pitch_bounds(sound)
    pitch = call(
        sound, "To Pitch", 0.0, pitch_floor, pitch_ceiling
    ) # check pitch to set formant settings
    mean_f0 = call(pitch, "Get mean", 0, 0, "Hertz")
    if 140 <= mean_f0 <= 300:
      max_formant = 5500
    
    elif mean_f0 < 140:
      max_formant = 5000
    
    else:
      max_formant = 8000
    return max_formant
  except:
    return 5500

def get_values_function (object, fn, command):
  if command == "Get mean":
    return call(object, command, fn, 0, 0, "hertz")
  elif command == "Get quantile":
    return call(object, command, fn, 0, 0, "hertz", 0.5)

def measure_formant(sound):
  # measure formants
  formant_args = {
      "time step": 0.0025, # a zero value is equal to 25% of the window length
      "max number of formants": 5.5, # must be divisible by 0.5
      "window length(s)": 0.025,
      "pre emphasis from": 50,
      # formant Burg-Specific settings
      "max_formant (To Formant Burg...)": ('Auto', ['Auto', '5500']),
      # formant Path-Specific settings
      "Center Formant (Formant Path)": ('Auto', ['Auto', '5500']),
      "Ceiling Step Size (Formant Path)": 0.05,
      'Number of Steps (Formant Path)': 4,
      'method': ('To Formant Burg...', ['To Formant Burg...', 'Formant Path'],),
  }

  # check that max number of formants is divisible by 0.5
  if formant_args['max number of formants'] % 0.5 != 0:
    # If it's not, round that to the nearest integer
    formant_args['max number of formants'] = round(formant_args['max number of formants'])

  formant_state = {
      "f1 means": [],
      "f2 means": [],
      "f3 means": [],
      "f4 means": [],
      "f1 medians": [],
      "f2 medians": [],
      "f3 medians": [],
      "f4 medians": [],
  }

  method = formant_args['method'][0]
  try:
    # Burg Method
    if method == 'To Formant Burg...':
      # Set Maximum Formant Value Automatically
      if formant_args['max_formant (To Formant Burg...)'] == 'Auto':
        try:
          formant_args['max_formant (To Formant Burg...)'] = formant_max(sound)
        except:
          # Otherwise default to 5500Hz as max_formant value, Praat's default
          formant_args['max_formant (To Formant Burg...)'] == 5500
      # Set any user defined max formant values
      else:
        try:
          formant_args['max_formant (To Formant Burg...)'] == int(formant_args['max_formant (To Formant Burg...)'])
        except:
          # Otherwise default to 5500Hz as max_formant value, Praat's default
          formant_args['max_formant (To Formant Burg...)'] = 5500

      # Create a Praat Formant Object
      formant_object = sound.to_formant_burg(
          formant_args["time step"],
          formant_args["max number of formants"],
          formant_args["max_formant (To Formant Burg...)"],
          formant_args["window length(s)"],
          formant_args["pre emphasis from"],
    )

    # measure_PCA = formant_args['Measure PCA']

      # Generate max_formant parameter
      formant_args["max_formant"] = formant_max(sound)
      formant_path_object = call(sound, "To FormantPath (burg)", 0.005, 5, formant_args["max_formant"], 0.025, 50.0, 0.025, 5)
      formant_object = call(formant_path_object, "Extract Formant")

      # formant_object = sound.to_formant_burg(
      #    self.args["time step"],
      #    self.args["max number of formants"],
      #    self.args["max_formant"],
      #    self.args["window length(s)"],
      #    self.args["pre emphasis from"],
      # )

      # Formant Path Method
    elif formant_args['method'] == 'Formant Path':
      try: # Try formant path first
        # Find centre max formant
        if formant_args['Center Formant (Formant Path)'] == 'Auto':
          try:
            formant_args['Center Formant (Formant Path)'] = formant_max(sound)
          except:
            # Otherwise default to 5500Hz as max_formant value, Praat's default
            formant_args['Center Formant (Formant Path)'] = 5500
        # Set any user defined max formant values
        else:
          try:
            formant_args['Center Formant (Formant Path)'] = int(
                formant_args['Center Formant (Formant Path)'])
          except:
            # Otherwise default to 5500 Hz as max_formant value, Praat's default
            formant_args['Center Formant (Formant Path)'] = 5500

        # Create A Praat Formant Path Object
        formant_path_object = call(sound,
                                   "To FormantPath (burg)",
                                   formant_args["time step"],
                                   formant_args["max number of formants"],
                                   formant_args["Center Formant (Formant Path)"],
                                   formant_args["window length(s)"],
                                   formant_args["pre emphasis from"],
                                   formant_args["Ceiling Step Size (Formant Path)"],
                                   formant_args["Number of Steps (Formant Path)"])
        # Extract the Praat Formant Object from the Formant Path Object
        formant_object = call(formant_path_object, "Extract Formant")
        # Reset the method value in case previous voice failed at formant path and instead,
        ## 'To Formant Burg...' was used
        formant_args["method"] = 'Formant Path'

      except:
        # If formant path fails, get formant object formant_burg with max_formant from formant_args
        formant_object = sound.to_formant_burg(
            formant_args["time step"],
            formant_args["max number of formants"],
            formant_args["max_formant (To Formant Burg...)"],
            formant_args["window length(s)"],
            formant_args["pre emphasis from"],
        )
      # Set the method to To formant burg to ensure we record the method change in the output data file
      formant_args["method"] = 'To formant burg...'

    # vectorise gathering the formants in Numpy to prevent nested loops
    # set up lists to apply the functions across, the list needs to be 8 items long
    # 4 Formants, mean and median ('Get quantile') = 8 items
    # 8 Formant objects
    objects = [formant_object] * 8
    # 8 formant numbers
    formant_number = [1, 2, 3, 4] * 2
    # 8 measures of central tendency
    command = ['Get mean'] * 4 + ['Get quantile'] * 4
    # creates a vectorised function from our get_values_function function
    get_values = np.frompyfunc(get_values_function, 3, 1)
    # runs the vectorised function and returns a list of formant means and medians
    formant_means_and_medians = get_values(objects, formant_number, command)
    # expands the list into the proper variable names to report the results
    f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median = formant_means_and_medians
    # reports the results
    results = {
        "F1 Mean": f1_mean,
        "F2 Mean": f2_mean,
        "F3 Mean": f3_mean,
        "F4 Mean": f4_mean,
        "F1 Median": f1_median,
        "F2 Median": f2_median,
        "F3 Median": f3_median,
        "F4 Median": f4_median,
    }

  except: 
    # If something went wrong beyond what's already been tested for, report nan values and move on
    results = {
        "F1 Mean": np.nan,
        "F2 Mean": np.nan,
        "F3 Mean": np.nan,
        "F4 Mean": np.nan,
        "F1 Median": np.nan,
        "F2 Median": np.nan,
        "F3 Median": np.nan,
        "F4 Median": np.nan,
    }     
  return results

In [5]:
def pitch_value(
    sound, floor=50, ceiling=500, method="ac", time_step=0, max_number_of_candidates=15,
    silence_threshold=0.03, voicing_threshold=0.45, octave_cost=0.01,
    octave_jump_cost=0.35, voiced_unvoiced_cost=0.14, unit="Hertz", very_accurate="no",
):
  pitch: object = call(
      sound, method, time_step, floor, max_number_of_candidates, very_accurate,
      silence_threshold, voicing_threshold, octave_cost, octave_jump_cost,
      voiced_unvoiced_cost, ceiling,
  )
  mean_f0: float = call(pitch, "Get mean", 0, 0, unit)
  stdev_f0: float = call(
      pitch, "Get standard deviation", 0, 0, unit
  )
  min_f0: float = call(pitch, "Get minimum", 0, 0, unit, "Parabolic")
  max_f0: float = call(pitch, "Get maximum", 0, 0, unit, "Parabolic")

  return pitch, mean_f0, stdev_f0, min_f0, max_f0

def measure_pitch(sound):
  pitch_args = {
      # simple default values
      "Time Step": 0, # positive float or 0
      "Max Number of Candidates": 15, # positive integers
      "Silence Threshold": 0.03, # positive number
      "Voicing Threshold": 0.45, # positive number
      "Octave Cost": 0.01, # positive number
      "Octave Jump Cost": 0.35, # positive number
      "Voiced Unvoiced Cost": 0.14, # positive number
      "Unit": ("Hertz", ["Hertz",
                        "Hertz (Logarithmic)",
                        "mel",
                        "logHertz",
                        "semitones re 1 Hz",
                        "semitones re 100 Hz",
                        "semitones re 200 Hz",
                        "semitones re 440 Hz",
                        "ERB",
                        ]
              ),
      # tuple defines a discrete set of options, 0 = selected value, 1 = a list of all options
      "Algorithm": ("To Pitch (ac)", ["To Pitch (ac)", "To Pitch (cc)"]),
      "Very Accurate": ("yes", ["yes", "no"]),
  }

  file_duration: float = call(sound, "Get total duration")
  
  time_step = pitch_args["Time Step"]
  max_number_of_candidates = pitch_args["Max Number of Candidates"]
  silence_threshold = pitch_args["Silence Threshold"]
  voicing_threshold = pitch_args["Voicing Threshold"]
  octave_cost = pitch_args["Octave Cost"]
  octave_jump_cost = pitch_args["Octave Jump Cost"]
  voiced_unvoiced_cost = pitch_args["Voiced Unvoiced Cost"]
  unit = pitch_args["Unit"][0]

  method = pitch_args["Algorithm"][0]
  very_accurate = pitch_args["Very Accurate"][0]

  try:
    pitch_floor, pitch_ceiling = pitch_bounds(sound)
    pitch, mean_f0, stdev_f0, min_f0, max_f0 = pitch_value(
        sound,
        floor=pitch_floor,
        ceiling=pitch_ceiling,
        method=method,
        time_step=time_step,
        max_number_of_candidates=max_number_of_candidates,
        silence_threshold=silence_threshold,
        voicing_threshold=voicing_threshold,
        octave_cost=octave_cost,
        octave_jump_cost=octave_jump_cost,
        voiced_unvoiced_cost=voiced_unvoiced_cost,
        unit=unit,
        very_accurate=very_accurate,
    )

    return {
        "Mean Pitch (F0)": mean_f0,
        "Standard Deviation Pitch (F0)": stdev_f0,
        "Pitch Min (F0)": min_f0,
        "Pitch Max (F0)": max_f0,
        "Pitch Floor": pitch_floor,
        "Pitch Ceiling": pitch_ceiling,
    }
  except:
    return {
        "Mean Pitch (F0)": "Pitch Measurement Failed",
        "Standard Deviation Pitch (F0)": "Pitch Measurement Failed",
        "Pitch Min (F0)": "Pitch Measurement Failed",
        "Pitch Max (F0)": "Pitch Measurement Failed",
        "Pitch Floor": "Pitch Measurement Failed",
        "Pitch Ceiling": "Pitch Measurement Failed",
    }

In [6]:
def manipulate_voice(file_path, sound, pitch, formant):
  args = {"unit": ("percent", ["percent"]),
          "formant_factor": formant,
          "pitch_factor": pitch,
          "pitch_range_factor": 1,
          "duration_factor": 1,
          "normalise amplitude": True
          }
  
  formant_factor = args["formant_factor"]
  pitch_factor = args["pitch_factor"]
  duration = sound.get_total_duration()
  pitch_range_factor = args["pitch_range_factor"]
  duration_factor = 1
  pitch_range_factor = 1
  f0min, f0max = pitch_bounds(sound)
  args['f0min'], args['f0max'] = f0min, f0max
  pitch = sound.to_pitch()
  median_pitch = call(pitch, "Get quantile", 0, duration, 0.5, "Hertz")

  new_pitch_median = pitch_factor * median_pitch

  output_file_name = file_path.split("/")[-1].split(".wav")[0]
  output_file_name = (
      f"{output_file_name}_lower_pitch_and_formants_{pitch_factor}_{formant_factor}"
  )
  number_of_channels = call(sound, 'Get number of channels')
  if number_of_channels == 2:
    sound = call(sound, 'Convert to mono')

  manipulated_sound = call(
      sound,
      "Change gender",
      f0min,
      f0max,
      formant_factor,
      new_pitch_median,
      pitch_range_factor,
      duration_factor,
  )

  if args["normalise amplitude"]:
    manipulated_sound.scale_intensity(70)

  manipulated_sound.name = output_file_name

  return manipulated_sound

In [7]:
from pydub import AudioSegment
import parselmouth

original = AudioSegment.from_mp3("/content/drive/MyDrive/졸업프로젝트/recordings/english29.mp3")
original.export("english29.wav", format="wav")
original_filename = "/content/drive/MyDrive/졸업프로젝트/english29.wav"
original_sound = parselmouth.Sound(original_filename)

print(measure_formant(original_sound))
print(measure_pitch(original_sound))

target = AudioSegment.from_mp3("/content/drive/MyDrive/졸업프로젝트/recordings/english14.mp3")
target.export("english14.wav", format="wav")
target_filename = "/content/drive/MyDrive/졸업프로젝트/english14.wav"
target_sound = parselmouth.Sound(target_filename)

print(measure_formant(target_sound))
print(measure_pitch(target_sound))

{'F1 Mean': 835.3309541951478, 'F2 Mean': 2024.0477285387035, 'F3 Mean': 3087.6912858403894, 'F4 Mean': 4156.46021212286, 'F1 Median': 747.9265385202275, 'F2 Median': 2094.3160122920017, 'F3 Median': 3120.1104640897465, 'F4 Median': 4259.80398874146}
{'Mean Pitch (F0)': 181.95097803469946, 'Standard Deviation Pitch (F0)': 31.700311835092094, 'Pitch Min (F0)': 129.36784743250442, 'Pitch Max (F0)': 492.1317171205534, 'Pitch Floor': 100, 'Pitch Ceiling': 500}
{'F1 Mean': 687.645677818269, 'F2 Mean': 1721.8399544922293, 'F3 Mean': 2656.797065048802, 'F4 Mean': 3651.264724700114, 'F1 Median': 591.52577307589, 'F2 Median': 1657.653781119284, 'F3 Median': 2529.5142230553265, 'F4 Median': 3544.1854787668685}
{'Mean Pitch (F0)': 138.12935666594854, 'Standard Deviation Pitch (F0)': 25.579287159553477, 'Pitch Min (F0)': 93.38145882150917, 'Pitch Max (F0)': 234.21403692467587, 'Pitch Floor': 50, 'Pitch Ceiling': 300}


In [8]:
# original

original_formant = measure_formant(original_sound)
original_pitch = measure_pitch(original_sound)

from IPython.display import Audio
Audio(data=original_sound.values, rate=original_sound.sampling_frequency)

Output hidden; open in https://colab.research.google.com to view.

In [9]:
# target

target_formant = measure_formant(target_sound)
target_pitch = measure_pitch(target_sound)

from IPython.display import Audio
Audio(data=target_sound.values, rate=target_sound.sampling_frequency)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
# target feature -> original
formant_factor = target_formant['F1 Mean'] / original_formant['F1 Mean']
formant_factor = formant_factor + target_formant['F2 Mean'] / original_formant['F2 Mean']
formant_factor = formant_factor + target_formant['F3 Mean'] / original_formant['F3 Mean']
formant_factor = formant_factor + target_formant['F4 Mean'] / original_formant['F4 Mean']
formant_factor = formant_factor / 4
print(formant_factor)

pitch_factor = target_pitch['Mean Pitch (F0)'] / original_pitch['Mean Pitch (F0)']
print(pitch_factor)

0.8531989944414524
0.7591569891952227


In [11]:
manipulated_sound = manipulate_voice(original_filename, original_sound, pitch_factor, formant_factor)
manipulated_formant = measure_formant(manipulated_sound)
manipulated_pitch = measure_pitch(manipulated_sound)
print(manipulated_formant, manipulated_pitch)

manipulated_sound.save("manipulated.wav", "WAV")
manipulated_filename = "/content/drive/MyDrive/졸업프로젝트/manipulated.wav"

from IPython.display import Audio
Audio(data=manipulated_sound.values, rate=manipulated_sound.sampling_frequency)

Output hidden; open in https://colab.research.google.com to view.

In [12]:
# original feature -> manipulated
formant_factor = original_formant['F1 Mean'] / manipulated_formant['F1 Mean']
formant_factor = formant_factor + original_formant['F2 Mean'] / manipulated_formant['F2 Mean']
formant_factor = formant_factor + original_formant['F3 Mean'] / manipulated_formant['F3 Mean']
formant_factor = formant_factor + original_formant['F4 Mean'] / manipulated_formant['F4 Mean']
formant_factor = formant_factor / 4
print(formant_factor)

pitch_factor = original_pitch['Mean Pitch (F0)'] / manipulated_pitch['Mean Pitch (F0)']
print(pitch_factor)

1.0989385728969778
1.3402128957591837


In [13]:
manipulated_sound = manipulate_voice(manipulated_filename, manipulated_sound, pitch_factor, formant_factor)
print(measure_formant(manipulated_sound))
print(measure_pitch(manipulated_sound))

from IPython.display import Audio
Audio(data=manipulated_sound.values, rate=manipulated_sound.sampling_frequency)

Output hidden; open in https://colab.research.google.com to view.

In [14]:
# original feature -> target
formant_factor = original_formant['F1 Mean'] / target_formant['F1 Mean']
formant_factor = formant_factor + original_formant['F2 Mean'] / target_formant['F2 Mean']
formant_factor = formant_factor + original_formant['F3 Mean'] / target_formant['F3 Mean']
formant_factor = formant_factor + original_formant['F4 Mean'] / target_formant['F4 Mean']
formant_factor = formant_factor / 4
print(formant_factor)

pitch_factor = original_pitch['Mean Pitch (F0)'] / target_pitch['Mean Pitch (F0)']
print(pitch_factor)

1.1727078140893823
1.3172506006433444


In [15]:
manipulated_sound = manipulate_voice(target_filename, target_sound, pitch_factor, formant_factor)
print(measure_formant(manipulated_sound))
print(measure_pitch(manipulated_sound))

from IPython.display import Audio
Audio(data=manipulated_sound.values, rate=manipulated_sound.sampling_frequency)

Output hidden; open in https://colab.research.google.com to view.